# Coefficient Table


This notebook loads, reshapes, and plots regression coefficients and CIs produced  
in `run_analysis.R`. It creates all of the regression tables in Extended Data.  

To run this file, you will need to download all of the code and data, which is 
available at:  
https://doi.org/10.7910/DVN/WANAX3  

## Setup

In [1]:
# ------------------------------------------------------------------------------

import os
import itertools
import pandas as pd
pd.set_option('display.width', 140)         # Set display width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', 50)   # Max single column width

# ------------------------------------------------------------------------------

# Set working directory to repository root
REPO_DIR = "google-exposure-engagement"
os.chdir(REPO_DIR); print(f"set working directory: {REPO_DIR}")

# Import custom functions
import sys
sys.path.insert(0, ".")
from functions import stringify, print_line, p_value_sig

set working directory: /home/rer/proj/meta-extension/analysis/replication/google-exposure-engagement


## Load Data

Load coefficients and 95% CI from regressions on 2018 and 2020 data produced by  
`run_analysis.R`

In [2]:
fp_coefficients = os.path.join("data", "tabs_stack.csv")
df = pd.read_csv(fp_coefficients, index_col=[0])
df.head()

,var,behavior,outcome,year,model,lower_without_query,coef_without_query,upper_without_query,p_without_query,t_without_query,lower_with_query,coef_with_query,upper_with_query,p_with_query,t_with_query,N
1,Age Group: 25-44,Search Results,Unreliable URLs,2018,Negative Binomial,-0.354063,0.127665,0.609393,0.602225,0.521838,-0.284664,0.169258,0.623179,0.463403,0.734384,94
2,Age Group: 45-64,Search Results,Unreliable URLs,2018,Negative Binomial,-0.506558,-0.023504,0.459549,0.923743,-0.095813,-0.402082,0.059761,0.521604,0.799051,0.254847,111
3,Age Group: 65+,Search Results,Unreliable URLs,2018,Negative Binomial,-0.172410,0.370039,0.912488,0.180360,1.343244,-0.127057,0.385578,0.898213,0.139771,1.481357,50
4,Not very strong Republican,Search Results,Unreliable URLs,2018,Negative Binomial,-1.571999,-0.732178,0.107643,0.087219,-1.716708,-1.261994,-0.503554,0.254885,0.192204,-1.307618,10
5,Ind/Lean Democrat,Search Results,Unreliable URLs,2018,Negative Binomial,-0.562427,0.025255,0.612938,0.932628,0.084620,-0.496470,0.038794,0.574059,0.886608,0.142744,23


## Format Data

In [3]:
# Clean variable names
df['var'] = (df['var']
               .str.replace("Democrat", "Dem.")
               .str.replace("Republican", "Rep.")
               .str.replace("Ind/", ""))

df['varname'] = df['var'].apply(
    lambda s: "Age Bin" if s.startswith("Age") else "7-Point Party ID")

mask = df['varname'] == 'Age Bin'
df.loc[mask, 'group'] = df.loc[mask, 'var'].str.split(': ', expand=True)[1]

mask = df['varname'] == '7-Point Party ID'
df.loc[mask, 'group'] = df.loc[mask, 'var']

# Rename values
df['type'] = df['behavior'].map({
    "Search Results": "Google Search Exposure", 
    "Follows from Search": "Google Search Follows",
    "Browser History": "Overall Engagement"
})


In [4]:
cols_index = ["year", "type", "varname", "group", "outcome", "N"]
measures = ['coef','lower','upper','p','t']
specs = ['without_query', 'with_query']
cols_values = [f"{m}_{s}" for m, s in itertools.product(measures, specs)]

melted = df.melt(id_vars=cols_index, value_vars=cols_values)
name_split = melted['variable'].str.split('_', n=1, expand=True)
name_split.columns = ["value_type", "query"]
tab = pd.concat([melted, name_split], axis=1)


cols_index = ["year", "type", "varname", "group", "query", "outcome", "N"]
columns=['value_type']
pivoted = tab.pivot_table(index=cols_index, values='value', columns=columns)
pivoted = pivoted.reset_index()
pivoted.columns.name = None
pivoted

,year,type,varname,group,query,outcome,N,coef,lower,p,t,upper
0,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Partisan Slant,23,0.000607,-0.050218,0.981238,0.023541,0.051433
1,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Unreliable URLs,23,0.038794,-0.496470,0.886608,0.142744,0.574059
2,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Partisan Slant,23,-0.013682,-0.069218,0.627989,-0.485144,0.041855
3,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Unreliable URLs,23,0.025255,-0.562427,0.932628,0.084620,0.612938
4,2018,Google Search Exposure,7-Point Party ID,Lean Rep.,with_query,Partisan Slant,11,0.093172,0.030706,0.003618,2.937798,0.155638
...,...,...,...,...,...,...,...,...,...,...,...,...
175,2020,Overall Engagement,Age Bin,25-44,without_query,Unreliable URLs,247,0.523244,-0.087783,0.093147,1.681405,1.134270
176,2020,Overall Engagement,Age Bin,45-64,without_query,Partisan Slant,226,0.053350,0.014319,0.007461,2.684068,0.092381
177,2020,Overall Engagement,Age Bin,45-64,without_query,Unreliable URLs,237,1.014400,0.398666,0.001277,3.234780,1.630134
178,2020,Overall Engagement,Age Bin,65+,without_query,Partisan Slant,128,0.068267,0.026153,0.001528,3.183103,0.110381


## P-value Correction

In [5]:
from statsmodels.stats.multitest import multipletests

def adjust_pvals(row, method='holm'):
    """Adjust P values across rows in a dataframe"""
    keys = row.index.tolist()
    pvals = row[keys]
    decisions, adj_pvals, sidak_aplha, bonf_alpha = multipletests(
        pvals=pvals, 
        alpha=0.05,
        method=method
    )
    output = dict(zip(keys, adj_pvals))
    return output

In [6]:
tab = pivoted.pivot(
    index=["year", "type", "varname", "group", "query"], 
    columns=['outcome'], 
    values=['p']
).stack(0)
display(tab.head())

# Apply adjustment and reshape output
adj = tab.apply(lambda row: adjust_pvals(row, method='holm'), axis=1)
adj = adj.apply(pd.Series)
adj = adj.stack().reset_index()
adj = adj.rename(columns={'level_6':'outcome', 0:'padj'})
adj = adj.drop(columns=adj.filter(regex="level_").columns)
display(adj.head())

# Merge adjusted P values 
cols_merge = ["year", "type", "varname", "group", "query", "outcome"]
out = pivoted.merge(adj, how='left', on=cols_merge)
out = out[cols_merge + ['N', 'coef','lower','upper','t','p','padj']]

# Show adjustment changes
print_line()
print("# Results that are not longer significant")
display(out[(
    (out['p'] < 0.05) & (out['padj'] >= 0.05) |
    (out['p'] < 0.05) & (out['padj'] >= 0.05)
)].round(4))

outcome                                                                            Partisan Slant  Unreliable URLs
year type                   varname          group                query                                           
2018 Google Search Exposure 7-Point Party ID Lean Dem.            with_query    p        0.981238         0.886608
                                                                  without_query p        0.627989         0.932628
                                             Lean Rep.            with_query    p        0.003618         0.785270
                                                                  without_query p        0.005877         0.581567
                                             Not very strong Dem. with_query    p        0.263618         0.318330

,year,type,varname,group,query,outcome,padj
0,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Partisan Slant,1.000000
1,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Unreliable URLs,1.000000
2,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Partisan Slant,1.000000
3,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Unreliable URLs,1.000000
4,2018,Google Search Exposure,7-Point Party ID,Lean Rep.,with_query,Partisan Slant,0.007236


--------------------------------------------------------------------------------
# Results that are not longer significant


,year,type,varname,group,query,outcome,N,coef,lower,upper,t,p,padj
40,2018,Google Search Follows,7-Point Party ID,Lean Rep.,with_query,Partisan Slant,12,0.0995,0.0027,0.1963,2.0270,0.0440,0.0880
58,2018,Google Search Follows,7-Point Party ID,Strong Rep.,without_query,Partisan Slant,63,0.0734,0.0058,0.1409,2.1407,0.0333,0.0667
68,2018,Google Search Follows,Age Bin,65+,with_query,Partisan Slant,44,0.0903,0.0070,0.1736,2.1367,0.0338,0.0677
72,2018,Overall Engagement,7-Point Party ID,Lean Dem.,without_query,Partisan Slant,24,-0.0996,-0.1902,-0.0090,-2.1622,0.0314,0.0628
84,2018,Overall Engagement,Age Bin,25-44,without_query,Partisan Slant,110,0.0854,0.0101,0.1607,2.2331,0.0263,0.0525
102,2020,Google Search Exposure,7-Point Party ID,Not very strong Rep.,with_query,Partisan Slant,39,0.0361,0.0045,0.0678,2.2431,0.0254,0.0508


In [7]:
# Format strings
tab = out.copy()

# Coefficients and P-values
tab['coef'] = stringify(tab['coef'], ".3f")
tab['t'] = stringify(tab['t'], ".2f")
tab['p'] = stringify(tab['p'], ".3f") + tab['p'].apply(p_value_sig)
tab['padj'] = stringify(tab['padj'], ".3f") + tab['padj'].apply(p_value_sig)

# CI string formatting
lower = stringify(tab['lower'], ".3f")
upper = stringify(tab['upper'], ".3f")
tab['ci'] = "("+lower+", "+upper+")"
tab['ci'] = tab['ci'].str.replace("(-, -)", "-", regex=False)

# Final output
tab['coef'] = tab['coef'] + " " + tab['ci']
tab

,year,type,varname,group,query,outcome,N,coef,lower,upper,t,p,padj,ci
0,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Partisan Slant,23,"0.001 (-0.050, 0.051)",-0.050218,0.051433,0.02,0.981,1.000,"(-0.050, 0.051)"
1,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Unreliable URLs,23,"0.039 (-0.496, 0.574)",-0.496470,0.574059,0.14,0.887,1.000,"(-0.496, 0.574)"
2,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Partisan Slant,23,"-0.014 (-0.069, 0.042)",-0.069218,0.041855,-0.49,0.628,1.000,"(-0.069, 0.042)"
3,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Unreliable URLs,23,"0.025 (-0.562, 0.613)",-0.562427,0.612938,0.08,0.933,1.000,"(-0.562, 0.613)"
4,2018,Google Search Exposure,7-Point Party ID,Lean Rep.,with_query,Partisan Slant,11,"0.093 (0.031, 0.156)",0.030706,0.155638,2.94,0.004**,0.007**,"(0.031, 0.156)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2020,Overall Engagement,Age Bin,25-44,without_query,Unreliable URLs,247,"0.523 (-0.088, 1.134)",-0.087783,1.134270,1.68,0.093,0.186,"(-0.088, 1.134)"
176,2020,Overall Engagement,Age Bin,45-64,without_query,Partisan Slant,226,"0.053 (0.014, 0.092)",0.014319,0.092381,2.68,0.007**,0.007**,"(0.014, 0.092)"
177,2020,Overall Engagement,Age Bin,45-64,without_query,Unreliable URLs,237,"1.014 (0.399, 1.630)",0.398666,1.630134,3.23,0.001**,0.003**,"(0.399, 1.630)"
178,2020,Overall Engagement,Age Bin,65+,without_query,Partisan Slant,128,"0.068 (0.026, 0.110)",0.026153,0.110381,3.18,0.002**,0.003**,"(0.026, 0.110)"


In [8]:
tab = tab.melt(
    id_vars=["year", "type", "varname", "group", "query", "outcome", "N"], 
    value_vars=['coef','lower','upper','t','p','padj']
)
tab

,year,type,varname,group,query,outcome,N,variable,value
0,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Partisan Slant,23,coef,"0.001 (-0.050, 0.051)"
1,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,with_query,Unreliable URLs,23,coef,"0.039 (-0.496, 0.574)"
2,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Partisan Slant,23,coef,"-0.014 (-0.069, 0.042)"
3,2018,Google Search Exposure,7-Point Party ID,Lean Dem.,without_query,Unreliable URLs,23,coef,"0.025 (-0.562, 0.613)"
4,2018,Google Search Exposure,7-Point Party ID,Lean Rep.,with_query,Partisan Slant,11,coef,"0.093 (0.031, 0.156)"
...,...,...,...,...,...,...,...,...,...
1075,2020,Overall Engagement,Age Bin,25-44,without_query,Unreliable URLs,247,padj,0.186
1076,2020,Overall Engagement,Age Bin,45-64,without_query,Partisan Slant,226,padj,0.007**
1077,2020,Overall Engagement,Age Bin,45-64,without_query,Unreliable URLs,237,padj,0.003**
1078,2020,Overall Engagement,Age Bin,65+,without_query,Partisan Slant,128,padj,0.003**


## Table: Regression Coefficients

In [9]:
label_order = [
   'Strong Democrat',
   'Not very strong Democrat',
   'Lean Democrat',
   'Lean Republican',
   'Not very strong Republican',
   'Strong Republican',]

label_order_short = [
   'Strong Dem.',
   'Not very strong Dem.',
   'Lean Dem.',
   'Lean Rep.',
   'Not very strong Rep.',
   'Strong Rep.',]

outcomes = ['Partisan Slant', 'Unreliable URLs']
var_list = ['coef', 't', 'padj']

for outcome in outcomes:
    for year in [2018, 2020]:

        _tab = tab.query("outcome == @outcome & "
                         "year == @year & "
                         "variable in @var_list").copy()
        _tab = _tab.pivot_table(
            index=['type', 'varname', 'group', 'N'], 
            columns=['year', 'query', 'variable'], 
            values=['value'], 
            aggfunc='first'
        ).fillna('-')


        # ----------------------------------------------------------------------
        # Columns

        _tab.columns = _tab.columns.droplevel(None) # Drop no-name level from pivot 
        categories = [
            {
                'values': _tab.columns.get_level_values('year'),
                'categories': [2018, 2020],
                'ordered': True
            },
            {
                'values': _tab.columns.get_level_values('query'),
                'categories': ['without_query', 'with_query'],
                'ordered': True
            },
            {
                'values': _tab.columns.get_level_values('variable'),
                'categories': var_list,
                'ordered': True
            },
        ]
        category_cols = [pd.Categorical(**category) for category in categories]
        _tab.columns = pd.MultiIndex.from_arrays(category_cols) 
        _tab.columns.names = [""]*len(_tab.columns.names) # Remove level names 
        _tab = _tab.sort_index(axis=1, level=[0, 1, 2])

        # ----------------------------------------------------------------------
        # Index 

        categories = [
            {
                'values': _tab.index.get_level_values('type'),
                'categories': ['Google Search Exposure', 
                               'Google Search Follows',
                               'Overall Engagement'],
                'ordered': True
            },
            {
                'values': _tab.index.get_level_values('varname'), 
                'categories': ['7-Point Party ID', 'Age Bin'],
                'ordered': True
            },
            {
                'values': _tab.index.get_level_values('group'),
                'categories': label_order_short + ['25-44', '45-64', '65+'],
                'ordered': True
            },
            {
                'values': _tab.index.get_level_values('N'),
                'ordered': False
            }
        ]
        category_cols = [pd.Categorical(**category) for category in categories]
        index_names = ['Type', 'Group', 'Label', 'N']
        _tab.index = pd.MultiIndex.from_arrays(category_cols, names=index_names)
        _tab = _tab.sort_index(axis=0, level=[0, 1, 2])

        # ----------------------------------------------------------------------
        # Save to latex file

        if outcome == "Partisan Slant":
            coef_type = "partisanship"
        elif outcome == "Unreliable URLs":
            coef_type = "unreliable"
            
        fp_table = f'outputs/tables/coefficients_{coef_type}_{year}.tex'
        _tab.style.to_latex(column_format='rlrrllll', buf=fp_table)
        print_line()
        print(f'saved: {fp_table}')
        display(_tab)


--------------------------------------------------------------------------------
saved: outputs/tables/coefficients_partisanship_2018.tex


2018                                                  \
                                                                            without_query                               with_query          
                                                                                     coef      t      padj                    coef      t   
Type                   Group            Label                N                                                                              
Google Search Exposure 7-Point Party ID Strong Dem.          98    -0.008 (-0.051, 0.035)  -0.35     0.730   0.001 (-0.039, 0.040)   0.03   
                                        Not very strong Dem. 26     0.013 (-0.039, 0.065)   0.50     0.878   0.027 (-0.021, 0.075)   1.12   
                                        Lean Dem.            23    -0.014 (-0.069, 0.042)  -0.49     1.000   0.001 (-0.050, 0.051)   0.02   
                                        Lean Rep.            11      0.097 (0.028, 0.165)   2.78    0.012*    0.093 (0.031, 0.156)   2.94   
                                        Not very strong Rep. 10     0.017 (-0.054, 0.088)   0.47     0.640   0.012 (-0.052, 0.077)   0.37   
                                        Strong Rep.          74     0.027 (-0.016, 0.071)   1.23     0.439   0.031 (-0.009, 0.071)   1.53   
                       Age Bin          25-44                93      0.067 (0.019, 0.116)   2.74    0.013*    0.085 (0.038, 0.131)   3.58   
                                        45-64                108     0.069 (0.020, 0.117)   2.80    0.011*    0.081 (0.034, 0.128)   3.41   
                                        65+                  49      0.094 (0.040, 0.147)   3.45   0.001**    0.115 (0.064, 0.166)   4.46   
Google Search Follows  7-Point Party ID Strong Dem.          90    -0.010 (-0.076, 0.055)  -0.31     0.806  -0.006 (-0.072, 0.060)  -0.17   
                                        Not very strong Dem. 26    -0.018 (-0.096, 0.061)  -0.45     0.655   0.013 (-0.063, 0.090)   0.34   
                                        Lean Dem.            19    -0.029 (-0.117, 0.059)  -0.65     1.000  -0.024 (-0.111, 0.062)  -0.55   
                                        Lean Rep.            12     0.068 (-0.032, 0.169)   1.34     0.181    0.100 (0.003, 0.196)   2.03   
                                        Not very strong Rep. 12     0.029 (-0.069, 0.127)   0.58     1.000   0.032 (-0.071, 0.136)   0.62   
                                        Strong Rep.          63      0.073 (0.006, 0.141)   2.14     0.067    0.085 (0.018, 0.151)   2.50   
                       Age Bin          25-44                90     0.056 (-0.018, 0.131)   1.50     0.272   0.069 (-0.006, 0.144)   1.81   
                                        45-64                95     0.071 (-0.003, 0.145)   1.88     0.123    0.095 (0.020, 0.171)   2.48   
                                        65+                  44     0.069 (-0.013, 0.152)   1.66     0.198    0.090 (0.007, 0.174)   2.14   
Overall Engagement     7-Point Party ID Strong Dem.          115  -0.097 (-0.164, -0.029)  -2.81    0.011*                       -      -   
                                        Not very strong Dem. 29    -0.065 (-0.149, 0.019)  -1.53     0.254                       -      -   
                                        Lean Dem.            24   -0.100 (-0.190, -0.009)  -2.16     0.063                       -      -   
                                        Lean Rep.            15     0.081 (-0.022, 0.183)   1.55     0.121                       -      -   
                                        Not very strong Rep. 17     0.035 (-0.063, 0.133)   0.70     0.967                       -      -   
                                        Strong Rep.          89      0.099 (0.029, 0.168)   2.80   0.005**                       -      -   
                       Age Bin          25-44                110     0.085 (0.010, 0.161)   2.23     0.053                       -      -

--------------------------------------------------------------------------------
saved: outputs/tables/coefficients_partisanship_2020.tex


2020                                                  \
                                                                            without_query                               with_query          
                                                                                     coef      t      padj                    coef      t   
Type                   Group            Label                N                                                                              
Google Search Exposure 7-Point Party ID Strong Dem.          130    0.007 (-0.019, 0.032)   0.51     1.000   0.004 (-0.021, 0.028)   0.30   
                                        Not very strong Dem. 72     0.003 (-0.024, 0.031)   0.22     1.000   0.004 (-0.023, 0.030)   0.26   
                                        Lean Dem.            69     0.002 (-0.025, 0.030)   0.16     1.000   0.002 (-0.025, 0.028)   0.12   
                                        Lean Rep.            19     0.017 (-0.027, 0.060)   0.76     0.901   0.023 (-0.018, 0.064)   1.09   
                                        Not very strong Rep. 39      0.041 (0.008, 0.074)   2.47    0.028*    0.036 (0.004, 0.068)   2.24   
                                        Strong Rep.          50     0.029 (-0.002, 0.060)   1.82     0.139   0.016 (-0.014, 0.046)   1.04   
                       Age Bin          25-44                166    0.018 (-0.009, 0.045)   1.30     0.363   0.013 (-0.013, 0.040)   0.99   
                                        45-64                151    0.026 (-0.002, 0.054)   1.80     0.145   0.018 (-0.009, 0.045)   1.32   
                                        65+                  86      0.048 (0.017, 0.078)   3.05   0.005**    0.036 (0.006, 0.066)   2.38   
Google Search Follows  7-Point Party ID Strong Dem.          97    -0.017 (-0.067, 0.033)  -0.67     1.000  -0.015 (-0.066, 0.035)  -0.59   
                                        Not very strong Dem. 50     0.008 (-0.047, 0.064)   0.30     1.000   0.011 (-0.046, 0.067)   0.37   
                                        Lean Dem.            51    -0.004 (-0.059, 0.051)  -0.15     1.000  -0.004 (-0.059, 0.051)  -0.14   
                                        Lean Rep.            14     0.023 (-0.062, 0.108)   0.53     1.000   0.039 (-0.048, 0.126)   0.88   
                                        Not very strong Rep. 27      0.081 (0.015, 0.147)   2.41    0.033*    0.081 (0.014, 0.148)   2.38   
                                        Strong Rep.          36      0.072 (0.010, 0.135)   2.27    0.048*    0.077 (0.012, 0.143)   2.32   
                       Age Bin          25-44                116    0.027 (-0.027, 0.080)   0.97     0.666   0.027 (-0.028, 0.082)   0.98   
                                        45-64                109    0.049 (-0.006, 0.103)   1.76     0.160   0.047 (-0.009, 0.103)   1.65   
                                        65+                  66      0.106 (0.047, 0.165)   3.52   0.001**    0.092 (0.030, 0.153)   2.93   
Overall Engagement     7-Point Party ID Strong Dem.          191   -0.031 (-0.065, 0.003)  -1.80     0.145                       -      -   
                                        Not very strong Dem. 106   -0.032 (-0.069, 0.005)  -1.68     0.186                       -      -   
                                        Lean Dem.            87   -0.054 (-0.093, -0.015)  -2.73    0.013*                       -      -   
                                        Lean Rep.            35      0.094 (0.041, 0.147)   3.48   0.001**                       -      -   
                                        Not very strong Rep. 59     0.026 (-0.018, 0.070)   1.16     0.245                       -      -   
                                        Strong Rep.          73      0.080 (0.038, 0.122)   3.71  0.000***                       -      -   
                       Age Bin          25-44                232    0.021 (-0.017, 0.058)   1.07     0.285                       -      -

--------------------------------------------------------------------------------
saved: outputs/tables/coefficients_unreliable_2018.tex


2018                                                 \
                                                                           without_query                              with_query          
                                                                                    coef      t     padj                    coef      t   
Type                   Group            Label                N                                                                            
Google Search Exposure 7-Point Party ID Strong Dem.          99   -0.391 (-0.877, 0.094)  -1.59    0.227  -0.337 (-0.775, 0.102)  -1.51   
                                        Not very strong Dem. 27   -0.229 (-0.812, 0.353)  -0.78    0.878  -0.265 (-0.786, 0.257)  -1.00   
                                        Lean Dem.            23    0.025 (-0.562, 0.613)   0.08    1.000   0.039 (-0.496, 0.574)   0.14   
                                        Lean Rep.            11    0.216 (-0.556, 0.989)   0.55    0.582  -0.103 (-0.843, 0.638)  -0.27   
                                        Not very strong Rep. 10   -0.732 (-1.572, 0.108)  -1.72    0.174  -0.504 (-1.262, 0.255)  -1.31   
                                        Strong Rep.          77    0.192 (-0.293, 0.677)   0.78    0.439   0.124 (-0.314, 0.562)   0.56   
                       Age Bin          25-44                94    0.128 (-0.354, 0.609)   0.52    0.602   0.169 (-0.285, 0.623)   0.73   
                                        45-64                111  -0.024 (-0.507, 0.460)  -0.10    0.924   0.060 (-0.402, 0.522)   0.25   
                                        65+                  50    0.370 (-0.172, 0.912)   1.34    0.180   0.386 (-0.127, 0.898)   1.48   
Google Search Follows  7-Point Party ID Strong Dem.          95   -0.279 (-0.934, 0.377)  -0.84    0.806  -0.449 (-1.100, 0.202)  -1.36   
                                        Not very strong Dem. 28   -0.427 (-1.206, 0.353)  -1.08    0.564  -0.640 (-1.397, 0.118)  -1.66   
                                        Lean Dem.            20   -0.269 (-1.086, 0.549)  -0.65    1.000  -0.276 (-1.060, 0.509)  -0.69   
                                        Lean Rep.            13    0.796 (-0.119, 1.711)   1.71    0.175   0.675 (-0.219, 1.569)   1.49   
                                        Not very strong Rep. 13   -0.345 (-1.361, 0.671)  -0.67    1.000  -0.286 (-1.306, 0.735)  -0.55   
                                        Strong Rep.          68    0.368 (-0.292, 1.029)   1.10    0.273   0.193 (-0.457, 0.842)   0.58   
                       Age Bin          25-44                94    0.307 (-0.342, 0.957)   0.93    0.352   0.252 (-0.387, 0.890)   0.78   
                                        45-64                100   0.258 (-0.402, 0.919)   0.77    0.441   0.215 (-0.436, 0.865)   0.65   
                                        65+                  49    0.423 (-0.319, 1.165)   1.12    0.262   0.304 (-0.411, 1.019)   0.84   
Overall Engagement     7-Point Party ID Strong Dem.          120  -0.060 (-0.834, 0.713)  -0.15    0.878                       -      -   
                                        Not very strong Dem. 32   -0.067 (-0.992, 0.858)  -0.14    0.887                       -      -   
                                        Lean Dem.            24    0.315 (-0.672, 1.303)   0.63    0.531                       -      -   
                                        Lean Rep.            15     1.894 (0.817, 2.970)   3.46  0.001**                       -      -   
                                        Not very strong Rep. 18    0.372 (-0.714, 1.458)   0.67    0.967                       -      -   
                                        Strong Rep.          91     1.268 (0.497, 2.039)   3.23  0.003**                       -      -   
                       Age Bin          25-44                117  -0.226 (-1.042, 0.589)  -0.55    0.585                       -      -   
                                        45-64        

--------------------------------------------------------------------------------
saved: outputs/tables/coefficients_unreliable_2020.tex


2020                                                  \
                                                                           without_query                               with_query          
                                                                                    coef      t      padj                    coef      t   
Type                   Group            Label                N                                                                             
Google Search Exposure 7-Point Party ID Strong Dem.          130   0.003 (-0.374, 0.379)   0.01     1.000   0.018 (-0.358, 0.394)   0.09   
                                        Not very strong Dem. 75    0.100 (-0.312, 0.512)   0.48     1.000   0.092 (-0.320, 0.503)   0.44   
                                        Lean Dem.            70    0.010 (-0.399, 0.420)   0.05     1.000   0.003 (-0.409, 0.414)   0.01   
                                        Lean Rep.            19   -0.188 (-0.872, 0.496)  -0.54     0.901  -0.209 (-0.896, 0.478)  -0.60   
                                        Not very strong Rep. 39    0.052 (-0.454, 0.558)   0.20     0.839   0.152 (-0.356, 0.660)   0.59   
                                        Strong Rep.          51   -0.098 (-0.576, 0.380)  -0.40     0.687   0.071 (-0.414, 0.557)   0.29   
                       Age Bin          25-44                169   0.270 (-0.127, 0.667)   1.34     0.363   0.292 (-0.109, 0.693)   1.43   
                                        45-64                152   0.157 (-0.255, 0.568)   0.75     0.455   0.230 (-0.188, 0.649)   1.08   
                                        65+                  88    0.104 (-0.345, 0.552)   0.45     0.650   0.139 (-0.325, 0.602)   0.59   
Google Search Follows  7-Point Party ID Strong Dem.          120  -0.060 (-1.084, 0.963)  -0.12     1.000   0.224 (-0.861, 1.310)   0.41   
                                        Not very strong Dem. 63   -0.035 (-1.113, 1.043)  -0.06     1.000   0.105 (-1.013, 1.224)   0.19   
                                        Lean Dem.            63    0.145 (-0.910, 1.200)   0.27     1.000   0.181 (-0.939, 1.301)   0.32   
                                        Lean Rep.            18    0.049 (-1.546, 1.644)   0.06     1.000   0.166 (-1.503, 1.835)   0.20   
                                        Not very strong Rep. 37    0.678 (-0.543, 1.898)   1.09     0.276   1.024 (-0.282, 2.330)   1.54   
                                        Strong Rep.          45    0.285 (-0.954, 1.523)   0.45     0.652   0.429 (-0.932, 1.789)   0.62   
                       Age Bin          25-44                154   0.464 (-0.693, 1.621)   0.79     0.666   0.730 (-0.511, 1.971)   1.16   
                                        45-64                138   0.859 (-0.292, 2.010)   1.47     0.160   1.258 (-0.014, 2.529)   1.94   
                                        65+                  83    0.942 (-0.274, 2.158)   1.52     0.129   1.201 (-0.150, 2.553)   1.75   
Overall Engagement     7-Point Party ID Strong Dem.          199   0.232 (-0.283, 0.747)   0.88     0.378                       -      -   
                                        Not very strong Dem. 112   0.255 (-0.316, 0.826)   0.88     0.381                       -      -   
                                        Lean Dem.            92    0.160 (-0.426, 0.747)   0.54     0.592                       -      -   
                                        Lean Rep.            36    0.644 (-0.137, 1.424)   1.62     0.106                       -      -   
                                        Not very strong Rep. 60    0.608 (-0.051, 1.267)   1.81     0.141                       -      -   
                                        Strong Rep.          76     1.141 (0.529, 1.752)   3.66  0.000***                       -      -   
                       Age Bin          25-44                247   0.523 (-0.088, 1.134)   1.68     0.186                       -      -   
                        